In [ ]:
!pip3 install mesa

     |████████████████████████████████| 668 kB 9.4 MB/s 
     |████████████████████████████████| 60 kB 6.4 MB/s 


In [ ]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd

import time
import datetime

In [ ]:
def obtener_habitacion(model):
    habitacion = np.zeros((model.grid.width, model.grid.height))
    for celda in model.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:
          if isinstance(contenido, Robot):
            habitacion[x][y] = 6
          else:
            habitacion[x][y] = contenido.estado

    return habitacion


class Robot(Agent):
    def __init__(self, id_unico, model):
        super().__init__(id_unico, model)
        self.siguientePosicion = None;
        self.caja = 0;
        self.siguienteCaja = None;
        

    def step(self):
      esta_celda = self.model.grid.get_cell_list_contents([self.pos])
      cajas = [obj for obj in esta_celda if isinstance(obj, Piso)]
      
      pasos_posibles = self.model.grid.get_neighborhood(
        self.pos,
        moore=False,
        include_center=False)
      
      for celda_vecina in pasos_posibles:
        robots = [obj for obj in self.model.grid.get_cell_list_contents([celda_vecina]) if isinstance(obj, Piso)][0]
        if robots.ocupado:
          pasos_posibles.remove(celda_vecina)
      
      self.siguientePosicion = self.pos;
      
      piso = [obj for obj in esta_celda if isinstance(obj, Piso)]
      piso[0].ocupado = False
      self.siguienteCaja = self.caja; 

        
      if self.caja == 0 and cajas[0].estado == 1 and (not cajas[0].apilar):
        self.siguienteCaja = 1
        cajas[0].estado = 0
      elif self.caja == 1:
        if cajas[0].apilar and (cajas[0].residuo == False and cajas[0].estado < 5 or  cajas[0].residuo == True and cajas[0].estado < self.model.numero_cajas%5):
          self.siguienteCaja = 0
          cajas[0].estado += 1
        else:
          self.siguientePosicion = self.random.choice(pasos_posibles)   
      else:
        self.siguientePosicion = self.random.choice(pasos_posibles)
          

    def advance(self):
      self.caja = self.siguienteCaja;
      self.model.grid.move_agent(self, self.siguientePosicion)
       
      
class Piso(Agent):
    def __init__(self, id_unico, model, estado, apilar, ocupado, residuo):
        super().__init__(id_unico, model)
        self.estado = estado
        self.apilar = apilar
        self.ocupado = ocupado
        self.residuo = residuo
       
      

            
class Habitaciones(Model):
    def __init__(self, n, m, X):
        self.numero_robots = 5
        self.numero_cajas = X
        self.grid = MultiGrid(n, m, True)
        self.schedule = SimultaneousActivation(self)
        estados = [0] * n * m
        indiceEstados = list(range(n * m))

        for i in range (self.numero_cajas):
          
          rand = self.random.choice(indiceEstados)
          estados[rand] = 1
          indiceEstados.remove(rand)
              
        
        for i in range (5):
            rand = self.random.choice(indiceEstados)
            estados[rand] = 6
            indiceEstados.remove(rand)

        
        cont = 0
        contApilar = 0;
        ApilacionesCompletas = int(X/5) + 1 
        for (content, x, y) in self.grid.coord_iter():
          estado = estados[cont]
          if estado == 6:
            estado = 0
            piso = Piso(cont, self, estado, False, True, False)
          elif estado == 1 and contApilar < ApilacionesCompletas:
            if contApilar == 0:
              piso = Piso(cont, self, estado, True, False ,True)
            else:
              piso = Piso(cont, self, estado, True, False ,False)
            contApilar += 1
          else:
            piso = Piso(cont, self, estado, False, False, False)
          self.grid.place_agent(piso, (x,y))
          self.schedule.add(piso)
          cont += 1
          
        robots = 0
        for (contenido, x, y) in self.grid.coord_iter(): 
          if estados[robots] == 6:
            robot = Robot(robots+cont, self)
            self.grid.place_agent(robot, (x,y))
            self.schedule.add(robot)
          robots += 1
        
        self.colectordatos = DataCollector(
            model_reporters={"Habitacion": obtener_habitacion})

    def cajasUnicas(model):
      contCajasUnicas = 0
      habitacion = np.zeros((model.grid.width, model.grid.height))
      for celda in model.grid.coord_iter():
          contenido_celda, x, y = celda
          for contenido in contenido_celda:
            if isinstance(contenido, Piso):
              if contenido.estado == 1:
                contCajasUnicas += 1
      return contCajasUnicas

    def cajasApiladas(model):
      contApilar = 0
      habitacion = np.zeros((model.grid.width, model.grid.height))
      for celda in model.grid.coord_iter():
          contenido_celda, x, y = celda
          for contenido in contenido_celda:
            if isinstance(contenido, Piso):
              if contenido.estado == 5:
                contApilar += 1
      return contApilar

    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()
        

In [ ]:
# Definimos el tamaño del Grid
N = 7
M = 7

# Definimos el número de cajas
X = 30

# Número de stacks completos
contApilacionesCompletas = int(X/5)

TIEMPO_MAXIMO_EJECUCION = 0.15

cont = 0

start_time = time.time()
model = Habitaciones(N, M, X)

while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and (model.cajasUnicas() > 0 or model.contApilar() != contApilacionesCompletas)):
    model.step()
    cont += 1

print('Tiempo de ordenamiento:', str(datetime.timedelta(seconds=(time.time() - start_time))))

print('Número de movimientos realizados:', cont)

Tiempo de ordenamiento: 0:00:00.150141
Número de movimientos realizados: 667


In [ ]:
todas_habitaciones = model.colectordatos.get_model_vars_dataframe()

In [ ]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(todas_habitaciones.iloc[0][0], cmap="Greys")

def animate(i):
    patch.set_data(todas_habitaciones.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(todas_habitaciones))

In [ ]:
anim

#Conclusiones
Algo que podría ayudar para que el problema sea más eficiente sería aumentar la cantidad de cajas que los robots pueden apilar, ya que al ser un número mínimo, los robots se tardan más al poder apilar las cajas, y por ende, se tarda más la simulación. Por esto mismo, como se puede ver en los resultados, los movimientos dados por los robots son se 449 movimientos. Al incrementar el número de cajas que se pueden apilar, los movimeintos de los robots se reducirían grandiosamente.